In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'stress-detection-from-social-media-articles:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4641718%2F7903032%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240423%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240423T145653Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2daa563ba907fad3eef5f3c7050ae89825cefa4d5205460c2e529b5e101f73b29c3ae24c27ef6e1f6157bb161c87f29e1c5fa8baf33868e765ab7a8e53ac53a6e540abbf605f5d6e4cc3d1971a216badc32cf691bb606150efcf3d35d1bd19678b6064fc82133d8d8ab126827c533309a5dd8c92f6c4376537597b5741844d898502e85f8378de69ba2aaf4b9981497bf4fd31de012cfb504ad1b681a154af073dfe7c5a4f079bf0a1b2f0571c735b34a6871132e67828c8b1efc0b962b567f481e40d05ffae08c47c3737709c1e3a59d9e6d3d0f6d42119f4222d69adc6c127d6e31573b1722d26bfb1f635326157755fe4755150d3f48c09fd132225bd521b'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 2385674 bytes downloaded
Downloaded and uncompressed: stress-detection-from-social-media-articles
Data source import complete.


In [2]:
import pandas as pd


# Loading Data

In [3]:
REDDIT = '/kaggle/input/stress-detection-from-social-media-articles/Twitter_Full.csv'
USECOLS = ['text', 'labels']
df = pd.read_csv(filepath_or_buffer=REDDIT, sep=';', usecols=USECOLS)
df.head()

,text,labels
0,Being s mom is cleaning 24/7 the same shit ove...,1
1,And now we have been given the walkthru book b...,0
2,Wishing YOU Peace Joy & Love! JoyTrain MentalH...,0
3,speak-no-evil monkey Can I Be Honest With You...,1
4,Psy Do u hv any regrets? Me No Psy Are you hap...,0


In [4]:
from plotly import express
express.pie(data_frame=df, names='labels', color='labels')

In [5]:
express.histogram(x=df['text'].str.len(), log_y=True)

# EDA

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8900 entries, 0 to 8899
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    8897 non-null   object
 1   labels  8900 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 139.2+ KB


In [7]:
df['labels'].value_counts()

labels
1    4534
0    4366
Name: count, dtype: int64

In [8]:
df.isnull().sum()

text      3
labels    0
dtype: int64

In [9]:
df.fillna('missing', inplace=True)
df.dropna(inplace=True)

# Creating A TF-IDF Model

In [ ]:
!pip install concrete-ml

In [11]:
import nltk
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from concrete.ml.sklearn import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [12]:
X=df['text']
y=df['labels']

In [13]:
express.scatter(data_frame=df, x='text', y='labels', color='labels', height=800, hover_name='text')

# spliting

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Model Training

In [21]:
clf = LogisticRegression(n_bits=8)
X_train_tfidf_dense = X_train_tfidf.toarray()
clf.fit(X_train_tfidf_dense, y_train)
X_test_tfidf_dense = X_test_tfidf.toarray()
y_pred = clf.predict(X_test_tfidf_dense)

In [28]:

print(clf.get_params())

{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_bits': 8, 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


In [22]:
accuracy = accuracy_score(y_test, y_pred)

# Testing

In [23]:
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.8398876404494382
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.80      0.83       856
           1       0.83      0.88      0.85       924

    accuracy                           0.84      1780
   macro avg       0.84      0.84      0.84      1780
weighted avg       0.84      0.84      0.84      1780



In [25]:
new_post = "Feeling overwhelmed with assignments"
new_post_tfidf_dense = tfidf_vectorizer.transform([new_post]).toarray()
prediction = clf.predict(new_post_tfidf_dense)
print("Predicted stress level:", prediction[0])

Predicted stress level: 1


In [26]:
new_post = "Congratulations on feeling great today!"
new_post_tfidf = tfidf_vectorizer.transform([new_post]).toarray()
prediction = clf.predict(new_post_tfidf)
print("Predicted stress level:", prediction[0])

Predicted stress level: 0
